In [10]:
import prepare_vocab
import train
import config
from data.loader import DataLoader
from utils import constant

In [13]:
# %Load glove vectors, get IDs for each token, lookup for each id in embedings and create embeding file for out dataset.
vocab_params = config.VocabParameters()
training_params = config.TrainingParameters()
opt = vars(vocab_params)
opt['num_class'] = len(constant.LABEL_TO_ID)
opt.update(vars(training_params))
vocab = prepare_vocab.prepare_voabulary (vocab_params)

loading files...
2525296 tokens from 68124 examples loaded from ./dataset/tacred/train.json.
802558 tokens from 22631 examples loaded from ./dataset/tacred/dev.json.
539009 tokens from 15509 examples loaded from ./dataset/tacred/test.json.
loading glove...
2195892 words loaded from glove.
building vocab...
vocab built with 55950/62152 words.
calculating oov...
train oov: 20546/2525296 (0.81%)
dev oov: 45801/802558 (5.71%)
test oov: 33634/539009 (6.24%)
building embeddings...
embedding size: 55950 x 300
dumping to files...
all done.


In [14]:
# load data
print("Loading data from {} with batch size {}...".format(vocab_params.data_dir, training_params.batch_size))
train_batch = DataLoader(vocab_params.data_dir+ '/train.json', training_params.batch_size, opt, vocab, evaluation=False)
dev_batch = DataLoader(vocab_params.data_dir + '/dev.json', training_params.batch_size, opt, vocab, evaluation=True)


Loading data from ./dataset/tacred with batch size 50...


NameError: name 'vocab' is not defined

In [ ]:
# Train model using Position Aware
train.train_model(vocab_params, training_params, train_batch, dev_batch, model_id='00')

# Train model using LSTM
train.train_model(vocab_params, training_params, train_batch, dev_batch, model_id='01')

1234